In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import chart_studio.plotly as ply
import os
from scipy import interpolate as interp
from scipy.signal import spectrogram, firwin, filtfilt, hilbert
from utilities.signal_analysis_methods import get_harmonic_envelope, get_total_envelope
import utm

In [ ]:
mag_path = 'data/mag_engi_7-3-25.csv'
gps_path = 'data/gps_engi_7-3-25.csv'

# Magnetic secondary coil data acquisition rate and primary coil driving frequency
sample_rate = int(2e3) # in [Hz]
driving_frequency = 100 # in [Hz]

mag_data = pd.read_csv(mag_path)
mag_times = mag_data['Times (s)'].to_numpy()[:sample_rate*500]
mag_ch1_voltages = mag_data['Secondary Ch 1 Voltage (V)'].to_numpy()[:sample_rate*500]
mag_ch2_voltages = mag_data['Secondary Ch 2 Voltage (V)'].to_numpy()[:sample_rate*500]
mag_primary_voltages = mag_data['Primary Voltage (V)'].to_numpy()[:sample_rate*500]
# AC voltages computed by subtracting means from raw voltages
mag_ch1_ac_voltages = mag_ch1_voltages-np.mean(mag_ch1_voltages)
mag_ch2_ac_voltages = mag_ch2_voltages-np.mean(mag_ch2_voltages)
mag_primary_ac_voltages = mag_primary_voltages-np.mean(mag_primary_voltages)

gps_data = pd.read_csv(gps_path)
gps_times = gps_data['Times (s)'].to_numpy()[:500]
gps_lat = gps_data['Latitude'].to_numpy()[:500]
gps_lon = gps_data['Longitude'].to_numpy()[:500]
gps_headings = gps_data['Heading'].to_numpy()[:500]
# Conversion from GPS global latitude and longitude to UTM latitude and longitude (in [m])
utm_lat, utm_lon, zone_num, zone_letter = utm.from_latlon(gps_lat, gps_lon)

Computation of RMS Voltages and Harmonic Envelopes

In [ ]:
# Computes RMS voltages 
num_rms_bins = int(len(mag_ch1_ac_voltages)/sample_rate)
mag_ch1_rms_voltages = [np.sqrt(np.mean(np.square(mag_ch1_ac_voltages[i*sample_rate:(i+1)*sample_rate]))).tolist() for i in range(num_rms_bins)]
mag_ch2_rms_voltages = [np.sqrt(np.mean(np.square(mag_ch2_ac_voltages[i*sample_rate:(i+1)*sample_rate]))).tolist() for i in range(num_rms_bins)]
mag_ch1_rms_voltages = mag_ch1_rms_voltages - np.mean(mag_ch1_rms_voltages)
mag_ch2_rms_voltages = mag_ch2_rms_voltages - np.mean(mag_ch2_rms_voltages)

# Computes harmonic envelopes
mag_ch1_fundamental_envelope = get_harmonic_envelope(mag_ch1_ac_voltages, sample_rate, mag_primary_ac_voltages, driving_frequency, 1)[2]
mag_ch1_fundamental_envelope = mag_ch1_fundamental_envelope - np.mean(mag_ch1_fundamental_envelope)
mag_ch2_fundamental_envelope = get_harmonic_envelope(mag_ch2_ac_voltages, sample_rate, mag_primary_ac_voltages, driving_frequency, 1)[2]
mag_ch2_fundamental_envelope = mag_ch2_fundamental_envelope - np.mean(mag_ch2_fundamental_envelope)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x= mag_times[::sample_rate], 
    y= mag_ch1_rms_voltages, 
    mode='lines', 
    name='Ch 1 RMS Voltages',
    line=dict(color='rebeccapurple', width=2)
))
fig.add_trace(go.Scatter(
    x= mag_times[1000:-1000],
    y= mag_ch1_fundamental_envelope[1000:-1000]-np.mean(mag_ch1_fundamental_envelope[1000:-1000]),   
    mode='lines', 
    name='Ch 1 Fundamental Envelope',
    line=dict(color='royalblue', width=2)
))
fig.update_layout(
    title='Ch 1 RMS Voltage and Fundamental Envelope vs. Time',
    xaxis_title='Time (s)',
    yaxis_title='Voltage (V)',
    template='plotly_dark',
    hovermode='x unified',
)
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x= gps_times, 
    y= gps_headings[1:]-gps_headings[:-1], 
    mode='lines', 
    name='GPS Heading Change',
    line=dict(color='rebeccapurple', width=2),
    showlegend=True
))
fig.update_layout(
    title='Heading Change vs. Time',
    xaxis_title='Time (s)',
    yaxis_title='Heading Change (degrees)',
    template='plotly_dark',
    hovermode='x unified',
)
fig.show()

Removing Turns from Magnetic Data

In [ ]:
for i in range(len(gps_headings)):
    if abs(gps_headings[i] - gps_headings[i-1]) > 5:
        # Removes data with some margin around the start and end of a turn
        mag_ch1_rms_voltages[i-3:i+3] = 0
        mag_ch2_rms_voltages[i-3:i+3] = 0
        mag_ch1_fundamental_envelope[int((i-3)*sample_rate):int((i+3)*sample_rate)] = 0
        mag_ch2_fundamental_envelope[int((i-3)*sample_rate):int((i+3)*sample_rate)] = 0

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x= mag_times[::sample_rate], 
    y= mag_ch1_rms_voltages, 
    mode='lines', 
    name='Ch 1 RMS Voltages',
    line=dict(color='rebeccapurple', width=2)
))
fig.add_trace(go.Scatter(
    x= mag_times[1000:-1000],
    y= mag_ch1_fundamental_envelope[1000:-1000]-np.mean(mag_ch1_fundamental_envelope[1000:-1000]),   
    mode='lines', 
    name='Ch 1 Fundamental Envelope',
    line=dict(color='royalblue', width=2)
))
fig.update_layout(
    title='Ch 1 RMS Voltage and Fundamental Envelope vs. Time',
    xaxis_title='Time (s)',
    yaxis_title='Voltage (V)',
    template='plotly_dark',
    hovermode='x unified',
)
fig.show()

Interpolation of RMS Voltages and Harmonic Envelopes to GPS Coordinates

In [ ]:
# Interpolates rms mag data to gps coords
mag_ch1_voltages_interp = interp.interp1d(mag_times[::sample_rate], mag_ch1_rms_voltages, kind='cubic', fill_value='extrapolate')
mag_ch2_voltages_interp = interp.interp1d(mag_times[::sample_rate], mag_ch2_rms_voltages, kind='cubic', fill_value='extrapolate')
mag_ch1_voltages_at_gps_coords = mag_ch1_voltages_interp(gps_times)
mag_ch2_voltages_at_gps_coords = mag_ch2_voltages_interp(gps_times)

# Interpolates fundamental frequency mag data to gps coords
mag_ch1_fundamental_envelope_interp = interp.interp1d(mag_times, mag_ch1_fundamental_envelope, kind='cubic', fill_value='extrapolate')
mag_ch1_fundamental_envelope_at_gps_coords = mag_ch1_fundamental_envelope_interp(gps_times)
mag_ch2_fundamental_envelope_interp = interp.interp1d(mag_times, mag_ch2_fundamental_envelope, kind='cubic', fill_value='extrapolate')
mag_ch2_fundamental_envelope_at_gps_coords = mag_ch2_fundamental_envelope_interp(gps_times)

Mapping of Secondary Coil Voltages and Complex Field Phase

In [ ]:
mag_map = plt.scatter(x=utm_lat[10:-10]%1000, y=utm_lon[10:-10]%1000, c=mag_ch1_fundamental_envelope_at_gps_coords[10:-10], cmap='viridis')
plt.colorbar(mag_map, label='Voltage (V)')
plt.xlabel('Latitude (m)')
plt.ylabel('Longitude (m)')
plt.title('Ch 1 Fundamental Envelope Map')
plt.axis('equal')
plt.grid(True)
plt.show()

mag_map = plt.scatter(x=utm_lat[10:-10]%1000, y=utm_lon[10:-10]%1000, c=mag_ch1_fundamental_envelope_at_gps_coords[10:-10], cmap='viridis')
plt.colorbar(mag_map, label='Voltage (V)')
plt.xlabel('Latitude (m)')
plt.ylabel('Longitude (m)')
plt.title('Ch 1 Fundamental Envelope Map')
plt.axis('equal')
plt.grid(True)
plt.show()

mag_map = plt.scatter(x=utm_lat%1000, y=utm_lon%1000, c=mag_ch1_voltages_at_gps_coords, cmap='viridis')
plt.colorbar(mag_map, label='Voltage (V)')
plt.xlabel('Latitude (m)')
plt.ylabel('Longitude (m)')
plt.title('Ch 1 RMS Voltage Map')
plt.axis('equal')
plt.grid(True)
plt.show()

mag_map = plt.scatter(x=utm_lat%1000, y=utm_lon%1000, c=mag_ch2_voltages_at_gps_coords, cmap='viridis')
plt.colorbar(mag_map, label='Ch 2 Voltage (V)')
plt.xlabel('Latitude (m)')
plt.ylabel('Longitude (m)')
plt.title('Ch 2 Voltage Map')
plt.axis('equal')
plt.grid(True)
plt.show()

mag_field_phase = np.arctan2(mag_ch2_voltages_at_gps_coords, mag_ch1_voltages_at_gps_coords)
mag_map = plt.scatter(x=utm_lat%1000, y=utm_lon%1000, c=mag_field_phase, cmap='viridis')
plt.colorbar(mag_map, label='arctan(Ch2/Ch1)')
plt.xlabel('Latitude (m)')
plt.ylabel('Longitude (m)')
plt.title('Phase Map')
plt.axis('equal')
plt.grid(True)
plt.show()

In [ ]:
logname = "arcgis_data.csv"
path = os.path.expanduser('data/'+logname)
logfile = open(path, "w")
logfile.write("Times (s),GPS Lat,GPS Lon,Ch 1 Voltage (V),Ch 2 Voltage (V),Phase\n")
for i in range(len(gps_times)):
        logfile.write(f"{gps_times[i]},{gps_lat[i]},{gps_lon[i]},\
                        {mag_ch1_voltages_at_gps_coords[i]:.7f},{mag_ch2_voltages_at_gps_coords[i]:.7f},{mag_field_phase[i]:.7f}\n")
logfile.close()